In [1]:
import pandas as pd
import pyodbc

# Kết nối đến SQL Server với Windows Authentication
connection_string = (
    "DRIVER={SQL Server};"
    "SERVER=LAPTOP-IBQB3FPP\MSSQLSERVER01;"
    "DATABASE=DataWarehouse1;"
    "Trusted_Connection=yes;"  # Sử dụng Windows Authentication
)



# Tạo kết nối
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# Đọc dữ liệu từ các file CSV
teams_df = pd.read_csv('data_clean/teams.csv')
seasons_df = pd.read_csv('data_clean/seasons.csv')
df_matches = pd.read_csv('data_clean/matches.csv')
team_stats_df = pd.read_csv('data_clean/team_stats.csv')

# Tạo bảng Teams
cursor.execute("""
IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'dbo.Teams') AND type in (N'U'))
BEGIN
    CREATE TABLE Teams (
        TeamID INT PRIMARY KEY,
        TeamName NVARCHAR(100) NOT NULL
    );
END
""")

# Tạo bảng Seasons
cursor.execute("""
IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'dbo.Seasons') AND type in (N'U'))
BEGIN
    CREATE TABLE Seasons (
        SeasonID INT PRIMARY KEY,
        Year NVARCHAR(10) NOT NULL
    );
END
""")

# Tạo bảng TeamStats
cursor.execute("""
IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'dbo.TeamStats') AND type in (N'U'))
BEGIN
    CREATE TABLE TeamStats (
        SeasonID INT,
        TeamID INT,
        Played INT,
        Won INT,
        Draw INT,
        Lost INT,
        GF INT,
        GA INT,
        GoalDifference INT,
        Points INT,
        PPG FLOAT,
        PercentPoints FLOAT,
        FOREIGN KEY (SeasonID) REFERENCES Seasons(SeasonID),
        FOREIGN KEY (TeamID) REFERENCES Teams(TeamID)
    );
END
""")

# Tạo bảng Matches
cursor.execute("""
IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'dbo.Matches') AND type in (N'U'))
BEGIN
    CREATE TABLE Matches (
        HomeTeamID INT,
        AwayTeamID INT,
        Date DATETIME,
        HomeScore FLOAT,
        AwayScore FLOAT,
        ShotsHome INT,
        ShotsAway INT,
        ShotsOnGoalHome INT,
        ShotsOnGoalAway INT,
        PassesHome INT,
        PassesAway INT,
        AccuratePassesHome INT,
        AccuratePassesAway INT,
        FoulsHome INT,
        FoulsAway INT,
        YellowCardsHome INT,
        YellowCardsAway INT,
        PossessionTimeHome FLOAT,
        PossessionTimeAway FLOAT,
        RedCardsHome INT,
        RedCardsAway INT,
        Label INT,
        FOREIGN KEY (HomeTeamID) REFERENCES Teams(TeamID),
        FOREIGN KEY (AwayTeamID) REFERENCES Teams(TeamID)
    );
END
""")

# 1. Đẩy dữ liệu vào bảng Teams
for index, row in teams_df.iterrows():
    cursor.execute("""
        IF NOT EXISTS (SELECT * FROM Teams WHERE TeamID = ?)
        BEGIN
            INSERT INTO Teams (TeamID, TeamName)
            VALUES (?, ?)
        END
    """, row['TeamID'], row['TeamID'], row['TeamName'])

# 2. Đẩy dữ liệu vào bảng Seasons
for index, row in seasons_df.iterrows():
    cursor.execute("""
        IF NOT EXISTS (SELECT * FROM Seasons WHERE SeasonID = ?)
        BEGIN
            INSERT INTO Seasons (SeasonID, Year)
            VALUES (?, ?)
        END
    """, row['SeasonID'], row['SeasonID'], row['Year'])

# 3. Đẩy dữ liệu vào bảng TeamStats
for index, row in team_stats_df.iterrows():
    cursor.execute("""
        IF NOT EXISTS (SELECT * FROM TeamStats WHERE SeasonID = ? AND TeamID = ?)
        BEGIN
            INSERT INTO TeamStats (SeasonID, TeamID, Played, Won, Draw, Lost, GF, GA, GoalDifference, Points, PPG, PercentPoints)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        END
    """, row['SeasonID'], row['TeamID'], row['SeasonID'], row['TeamID'], row['Played'], 
           row['Won'], row['Draw'], row['Lost'], row['GF'], row['GA'], 
           row['Goal Difference'], row['Points'], row['PPG'], row['% Points'])

# 4. Đẩy dữ liệu vào bảng Matches
for index, row in df_matches.iterrows():
    cursor.execute("""
        IF NOT EXISTS (SELECT * FROM Matches WHERE HomeTeamID = ? AND AwayTeamID = ? AND Date = ?)
        BEGIN
            INSERT INTO Matches (HomeTeamID, AwayTeamID, Date, HomeScore, AwayScore, ShotsHome, ShotsAway, 
            ShotsOnGoalHome, ShotsOnGoalAway, PassesHome, PassesAway, AccuratePassesHome, AccuratePassesAway, 
            FoulsHome, FoulsAway, YellowCardsHome, YellowCardsAway, PossessionTimeHome, PossessionTimeAway, 
            RedCardsHome, RedCardsAway, Label)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        END
    """, row['HomeTeamID'], row['AwayTeamID'], row['date'],
           row['HomeTeamID'], row['AwayTeamID'], row['date'], 
           row['home_score'], row['away_score'], row['shots_home'], 
           row['shots_away'], row['shots_on_goal_home'], 
           row['shots_on_goal_away'], row['passes_home'], 
           row['passes_away'], row['accurate_passes_home'], 
           row['accurate_passes_away'], row['fouls_home'], 
           row['fouls_away'], row['yellow_cards_home'], 
           row['yellow_cards_away'], row['possession_time_home'], 
           row['possession_time_away'], row['red_cards_home'], 
           row['red_cards_away'], row['label'])

# Commit các thay đổi
conn.commit()

# Đóng kết nối
cursor.close()
conn.close()

print("Dữ liệu đã được đổ vào SQL Server thành công!")


<>:7: SyntaxWarning: invalid escape sequence '\M'
<>:7: SyntaxWarning: invalid escape sequence '\M'
C:\Users\ACER USER\AppData\Local\Temp\ipykernel_8820\1534896747.py:7: SyntaxWarning: invalid escape sequence '\M'
  "SERVER=LAPTOP-IBQB3FPP\MSSQLSERVER01;"


Dữ liệu đã được đổ vào SQL Server thành công!


In [4]:
import pandas as pd
import pyodbc

# Kết nối đến SQL Server
# Kết nối đến SQL Server với Windows Authentication
connection_string = (
    "DRIVER={SQL Server};"
    "SERVER=LAPTOP-IBQB3FPP\MSSQLSERVER01;"
    "DATABASE=DataWarehouse1;"
    "Trusted_Connection=yes;"  # Sử dụng Windows Authentication
)

# Tạo kết nối
conn = pyodbc.connect(connection_string)

# Truy vấn dữ liệu từ bảng Matches
query = """
SELECT *
FROM Matches m
JOIN Teams t1 ON m.HomeTeamID = t1.TeamID
JOIN Teams t2 ON m.AwayTeamID = t2.TeamID
"""

# Đọc dữ liệu vào DataFrame
df_matches = pd.read_sql_query(query, conn)

# Đóng kết nối
conn.close()

<>:8: SyntaxWarning: invalid escape sequence '\M'
<>:8: SyntaxWarning: invalid escape sequence '\M'
C:\Users\ACER USER\AppData\Local\Temp\ipykernel_8820\248102300.py:8: SyntaxWarning: invalid escape sequence '\M'
  "SERVER=LAPTOP-IBQB3FPP\MSSQLSERVER01;"
C:\Users\ACER USER\AppData\Local\Temp\ipykernel_8820\248102300.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_matches = pd.read_sql_query(query, conn)


In [5]:
df_matches

,HomeTeamID,AwayTeamID,Date,HomeScore,AwayScore,ShotsHome,ShotsAway,ShotsOnGoalHome,ShotsOnGoalAway,PassesHome,...,YellowCardsAway,PossessionTimeHome,PossessionTimeAway,RedCardsHome,RedCardsAway,Label,TeamID,TeamName,TeamID,TeamName
0,3,4,2019-08-11,4.0,0.0,11,18,5,7,449,...,4,0.46,0.54,0,0,1,3,Manchester United,4,Chelsea
1,5,7,2019-08-11,0.0,0.0,15,8,1,2,666,...,2,0.70,0.30,0,0,0,5,Leicester City,7,Wolverhampton Wanderers
2,13,8,2019-08-11,0.0,1.0,9,8,2,2,380,...,3,0.38,0.62,0,0,-1,13,Newcastle United,8,Arsenal
3,6,17,2019-08-10,3.0,1.0,31,7,7,4,587,...,0,0.69,0.31,0,0,1,6,Tottenham Hotspur,17,Aston Villa
4,18,9,2019-08-10,1.0,1.0,13,8,3,3,487,...,1,0.53,0.47,0,0,0,18,AFC Bournemouth,9,Sheffield United
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1887,14,17,2024-05-11,5.0,0.0,15,8,9,2,552,...,4,0.54,0.46,0,0,1,14,Crystal Palace,17,Aston Villa
1888,1,7,2024-05-11,2.0,0.0,36,4,14,3,621,...,1,0.67,0.33,0,1,1,1,Liverpool,7,Wolverhampton Wanderers
1889,26,22,2024-05-11,2.0,4.0,15,16,6,8,304,...,4,0.40,0.60,0,0,-1,26,Luton Town,22,Fulham
1890,2,16,2024-05-11,3.0,1.0,28,3,12,2,782,...,1,0.71,0.29,0,0,1,2,Manchester City,16,West Ham United
